# Атака на ошибки при работе RSA CRT (КТО)
Поскольку у RSA большое энергопотребление и он требует большой вычислительной мощности, на слабых устройствах часто используется вариант расшифрования на основе Китайской Теоремы об Остатках.
Предвариельно вычисляются значения $dP=e^{-1}\space mod \space (p-1)$, $dQ=e^{-1}\space mod \space (q-1)$ и $qInv=q^{-1}\space mod \space p$.
После этого алгоритм расшифрования работает следующим образом:
$$M_p=C^{dP}\space mod \space p$$
$$M_q=C^{dQ}\space mod \space q$$
$$h=qInv\cdot(M_p-M_q)\space mod\space p$$
$$M=M_q+h\cdot q$$
$M_p=M\space mod \space p$ and $M_q=M\space mod\space p$

Алгоритм отлично работает, сильно уменьшая вычислительную сложность RSA, но у него есть один недостаток. Если внедрить ошибку(фолт) при вычислении одного из остатков $M$, то получится $M'$, для которого:
$$M'_p=M_p$$
$$M'_q\ne M_q$$
Раз $M'_p=M_p$, то $M-M'=kp$, для некоторого $k \in \mathbb{Z}$, и $GCD(M-M',N)=p$, что позволяет факторизовать $N$ и вычислить $d$.

Получите с сервера $M$, $M'$, вычислите $d$ и отправьте на сервер, чтобы получить флаг. Удачи!

In [1]:
import socket
import re
from Crypto.Util.number import bytes_to_long,long_to_bytes,inverse,GCD
class VulnServerClient:
    def __init__(self,show=True):
        """Инициализация, подключаемся к серверу"""
        self.s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.s.connect(('cryptotraining.zone',1340))
        if show:
            print (self.recv_until().decode())
    def recv_until(self,symb=b'\n>'):
        """Получаем сообщения с сервера, по дефолту до знака приглашения"""
        data=b''
        while True:
            
            data+=self.s.recv(1)
            if data[-len(symb):]==symb:
                break
        return data
    def get_public_key(self,show=True):
        """Получаем открытый ключ с сервера"""
        self.s.sendall('public\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        e=int(re.search('(?<=e: )\d+',response).group(0))
        N=int(re.search('(?<=N: )\d+',response).group(0))
        self.num_len=len(long_to_bytes(N))
        return (e,N)
    
    def decryptBytes(self,m,show=True):
        """Получить окрытый текст от выбранного шифротекста в байтах с сервера"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        if len(m)>num_len:
            print ("The message is too long")
            return
        if len(m)<num_len:
            m=bytes((num_len-len(m))*[0x0])+m
        hex_m=m.hex().encode()
        self.s.sendall(b'decrypt '+hex_m+b'\n')
        response=self.recv_until().decode()
        if show:
            print (response)
        if response.find('flag')!=-1:
            print('You tried to submit \'flag\'')
            return None
        signature_hex=re.search('(?<=Signature: )[0-9a-f]+',response).group(0)
        signature_bytes=bytes.fromhex(signature_hex)
        return bytes_to_long(signature_bytes)
    
    
    def decryptNumber(self,m,show=True):
        """Получить открытый текст для выбранного закрытого текста в виде числа с сервера"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        return self.decryptBytes(long_to_bytes(m,num_len),show)
    
    def faultyDecryptBytes(self,m,show=True):
        """Получить открытый текст с ошибкой в алгоритме для выбранного закрытого текста в байтах с сервера"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        if len(m)>num_len:
            print ("The message is too long")
            return
        if len(m)<num_len:
            m=bytes((num_len-len(m))*[0x0])+m
        hex_m=m.hex().encode()
        self.s.sendall(b'faulty_decrypt '+hex_m+b'\n')
        response=self.recv_until().decode()
        if show:
            print (response)
        if response.find('flag')!=-1:
            print('You tried to submit \'flag\'')
            return None
        signature_hex=re.search('(?<=Signature: )[0-9a-f]+',response).group(0)
        signature_bytes=bytes.fromhex(signature_hex)
        return bytes_to_long(signature_bytes)
    
    
    def faultyDecryptNumber(self,m,show=True):
        """Получить открытый текст с ошибкой в алгоритме для выбранного закрытого текста в числовом представлении с сервера"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        return self.faultyDecryptBytes(long_to_bytes(m,num_len),show)
        
    def checkDNumber(self,c,show=True):
        """Проверить, является ли это число d"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        signature_bytes=long_to_bytes(c,num_len)
        self.checkDBytes(signature_bytes,show)
    
    def checkDBytes(self,c,show=True):
        """Проверить, является ли эта последовательность байтов d"""
        try:
            num_len=self.num_len
        except KeyError:
            print ('You need to get the public key from the server first')
            return
        if len(c)>num_len:
            print ("The message is too long")
            return
        
        hex_c=c.hex().encode()
        self.s.sendall(b'flag '+hex_c+b'\n',)
        response=self.recv_until(b'\n').decode()
        
        if show:
            print (response)
        
        if response.find('Wrong')!=-1:
            print('Wrong signature')
            x=self.recv_until()
            if show:
                print (x)
            return
        flag=re.search('CRYPTOTRAINING\{.*\}',response).group(0)
        print ('FLAG: ',flag)
        
    def __del__(self):
        self.s.close()


In [2]:
vs=VulnServerClient()
(e,N)=vs.get_public_key()

Welcome to RSA CRT Decryption Faults task
Available commands:
help - print this help
public - show public key
decrypt <hex(data)> - decrypt ciphertext
faulty_decrypt <hex(data)> - decrypt with fault
flag <hex(d))> - print flag 
quit - quit
>
e: 65537
N: 20159717663186764200842482638329142432479376755681286432561400011207751568770239378735042390550988864636478212097889382541806378632813451522011734778394352464750695430236459156439656932108536936107092785759187120915559173321302027525229018106368725032056109022369913503577023942696069608771010384365856481001383579432844112231215767630328627015097422540087789462404508697086321213990868031273219614897901436844999442259387453021270642395531884848697650933478124254071912232445708062597679170291021925633789812405697682134528381868778865376836541179591638312152472136313757252384761293684336082840137773984575947459061
>
